# **Letterboxd Rating SQL Analysis**

I've rated many dozens of films in [**Letterboxd**](https://letterboxd.com), the film rating and review social platform. But with all of this meticulous cataloguing, I'm curious to know if my ratings can teach me a few things about my film-enjoying habits:

**1\. Am I rating films regularly and reliably?**

**2\. How do my film ratings compare with the Letterboxd community at large?**

**3\. Can my ratings reveal characteristics of films that I should seek out — or avoid — when finding new films to watch?**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Below, I employ <b>PostgreSQL</b> queries to answer these questions and get a stronger understanding of my film taste.</span>

All tables used were created by importing CSV files into pgAdmin on my desktop. I downloaded the data for myratings directly from my Letterboxd account, each row comprising a film title, the year of its release, its Letterboxd page URL, the day of my rating, and my rating out of five stars. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">I web scraped the 171 Letterboxd film pages for the films I had rated to source the data in the&nbsp;</span>  <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">actors, directors, genres, lbrating (Letterboxd universal user rating), length, ratingcount, and studios tables. Each row in these tables lists the film ID (derived from its unique Letterboxd URL), film title, and associated variable of interest.</span> [**See my web scraping with Python here.**](https://github.com/andrewdkim7/portfolio/blob/c1bfea132a942d81b43c5a08284fb7a669eaa10b/Python/LetterboxdWebScraping.ipynb)

## But first, some data cleaning.

In order to join my myratings table with others, I create a uniform url\_id variable — the last three to four characters in each film's Letterboxd page URL.

In [355]:
ALTER TABLE myratings
ADD COLUMN IF NOT EXISTS url_id VARCHAR;

UPDATE myratings
SET url_id = SPLIT_PART(url, '/', 4);

NOTICE: column "url_id" of relation "myratings" already exists, skipping

ALTER TABLE

UPDATE 171

Total execution time: 00:00:00.033

## **Question #1:** **Am I a consistent and reliable film watcher/rater?**

First, I need to know if I watching and rating films well <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">enough to justify my instantaneous rush to the Letterboxd app every time I reach the end credits of a film.</span>

### **Summary statistics of my film ratings:**

In [356]:
WITH myratingmean (mean) AS (
    SELECT ROUND(AVG(myrating)::NUMERIC, 2)
    FROM myratings
    ),
myratingmed (median) AS (
    SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY myrating)
    FROM myratings
    ),
myratingmode (mode) AS (
    SELECT myrating
    FROM myratings
    GROUP BY myrating
    ORDER BY COUNT(myrating) DESC
    LIMIT 1
    ),
myratingrange (max, min, range) AS (
    SELECT MAX(myrating), 
    MIN(myrating), 
    (MAX(myrating) - MIN(myrating))
    FROM myratings
    ), 
myratingvar (variance) AS (
    SELECT ROUND((SUM(POWER(my.myrating - mya.mean, 2)) / (COUNT(my.*) - 1))::NUMERIC, 2)
    FROM myratings my, myratingmean mya
    ),
myratingsd (std_dev) AS (
    SELECT ROUND(SQRT(variance)::NUMERIC, 2)
    FROM myratingvar
    )
SELECT *
FROM myratingmean, myratingmed, myratingmode, myratingrange, myratingvar, myratingsd;


(1 row(s) affected)

Total execution time: 00:00:00.016

mean,median,mode,max,min,range,variance,std_dev
3.44,3.5,4,5,0.5,4.5,1.02,1.01


In [357]:
-- distribution of my film ratings in buckets
SELECT CONCAT(FLOOR(myrating), ' - ', 
    CASE
        WHEN FLOOR(myrating) = 5 THEN 5.0
        ELSE FLOOR(myrating) + .9
    END) bucket, 
    COUNT(*), 
    ROUND((COUNT(*)::NUMERIC / (SELECT COUNT(*) FROM myratings)), 2) share
FROM myratings
GROUP BY FLOOR(myrating)
ORDER BY FLOOR(myrating);

(6 row(s) affected)

Total execution time: 00:00:00.039

bucket,count,share
0 - 0.9,1,0.01
1 - 1.9,13,0.08
2 - 2.9,24,0.14
3 - 3.9,51,0.30
4 - 4.9,73,0.43
5 - 5,9,0.05


The mean film in my ratings received a 3.44 stars from me, although the bucketed distribution of my film ratings seems to indicate that I give significantly more films a rating 4 stars and above than I do below 3 stars. How generous of me!

<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Although my ratings are not normally distributed (the distribution demonstrates a skew left, in favor of higher ratings), the clustering of ratings around the median on the distribution table suggests I can interpret the standard deviation of my ratings, about 1.01 stars, to indicate a large majority of films I rate receive between approximately 2.5 and 4.5 stars. The distribution table illustrates that 73 percent of films I rated got a 3, 3.5, 4, or 4.5.</span>

<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;"><b>Conclusion:</b>&nbsp;Some combination of "most films I watch are 'pretty good'!" and "I should be a little more critical of the films I watch"</span>

### **How consistently do I watch and rate new films?**

In [358]:
-- number of films watched per month
SELECT EXTRACT(MONTH FROM date) AS watch_month, 
    EXTRACT(YEAR FROM date) AS watch_year, 
    COUNT(*) watch_count
FROM myratings
GROUP BY EXTRACT(MONTH FROM date), EXTRACT(YEAR FROM date)
ORDER BY watch_year, watch_month;

(13 row(s) affected)

Total execution time: 00:00:00.051

watch_month,watch_year,watch_count
5,2022,1
6,2023,115
7,2023,2
8,2023,4
9,2023,15
10,2023,11
11,2023,8
12,2023,3
1,2024,1
2,2024,4


The spikes in my film-watching rates make perfect sense when I think back to my account history. In May 2022, I opened my Letterboxd account at the pleading of my friends and added only one film: Parasite (2016) (5 stars). It wasn't until June 2023 that I decided to take the film-rating more seriously and reviewed every single film I could remember watching up until that point in life in a single day.

In the fall semester of my junior year of college, I took a Latin American Cinema class, which explains the frequency of my film ratings from August through November 2023. The following spring, I was the busiest I've been in my life, accounting for the dismal one film a month during that period. Fortunately, it l<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">ooks like the dawn of summer break in May 2024 has restored me back to a film-watching rate I want to be at!</span>

In [359]:
-- average time in between watches
WITH watch_distances (days_between) AS (
    SELECT date - (LAG(date, 1) OVER(ORDER BY date ASC))
    FROM myratings
    WHERE date > '2023-06-13'
    )
SELECT ROUND(AVG(days_between)::NUMERIC, 2) avg_days_between
FROM watch_distances;

(1 row(s) affected)

Total execution time: 00:00:00.282

avg_days_between
6.06


Discounting the period before I was logging films on Letterboxd, I watched one new film about every six days!

**Conclusion:** I should take more time to chill out and watch films during the school year.

### **Do my ratings get more or less generous over time?**

For each film, I extract the month and year I watched it and a rating generosity score, which I operationalize as <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">the difference between my rating for the film and its overall average rating from all Letterboxd members. I start from June 2023, when I started consistently reviewing films on the app.</span>

In [360]:
-- month/year watched and monthly average difference in my ratings from the community
WITH sample_dates (watch_month, watch_year, rating_diff) AS (
    SELECT EXTRACT(month FROM my.date), 
        EXTRACT(year FROM my.date),
        (my.myrating - lb.lb_rating)
    FROM myratings my JOIN lbrating lb ON my.url_id = lb.url_id
    WHERE date >= '2023-06-12'
    )
SELECT watch_month, 
    watch_year, 
    ROUND(AVG(rating_diff)::NUMERIC, 2) avg_difference
FROM sample_dates sd
GROUP BY watch_year, watch_month
ORDER BY watch_year, watch_month;

(12 row(s) affected)

Total execution time: 00:00:00.037

watch_month,watch_year,avg_difference
6,2023,-0.06
7,2023,-0.01
8,2023,-0.14
9,2023,-0.97
10,2023,-0.95
11,2023,-0.34
12,2023,-0.01
1,2024,0.74
2,2024,-0.51
3,2024,0.74


This relation seems to fluctuate a bit, trending negatively in late 2023 then in the positive in 2024, but I want to perform a linear regression to be certain whether a long-term change in film-rating generosity over time is statistically significant:

In [361]:
-- months since I started on Letterboxd and the differences in my and community ratings
CREATE OR REPLACE VIEW months_passed AS
    SELECT (EXTRACT(month FROM my.date) - 6) + ((EXTRACT(year FROM my.date) - 2023) * 12) months,
        (my.myrating - lb.lb_rating) rating_diff
    FROM myratings my JOIN lbrating lb ON my.url_id = lb.url_id
    WHERE date >= '2023-06-12';

-- linear regression of ratings differences on months I have been using Letterboxd
SELECT ROUND(REGR_SLOPE(rating_diff, months)::NUMERIC, 2) slope, 
    ROUND(REGR_INTERCEPT(rating_diff, months)::NUMERIC, 2) intercept
FROM months_passed;

-- t-test for significance of months coefficient, assuming normal sampling distribution given n >= 30
WITH regression (beta, alpha, sxx) AS (
    SELECT REGR_SLOPE(rating_diff, months), 
        REGR_INTERCEPT(rating_diff, months),
        REGR_SXX(rating_diff, months)
    FROM months_passed
    ),
mean_squared_error (mse) AS (
    SELECT SUM(rss.res_sum_squares) / (COUNT(*) - 2)
    FROM (
        SELECT POWER(mp.rating_diff - (r.beta * mp.months + r.alpha), 2) res_sum_squares
        FROM months_passed mp, regression r
        ) rss
    )
SELECT ROUND((r.beta / SQRT(m.mse / r.sxx))::NUMERIC, 2) t_stat
FROM regression r, mean_squared_error m
GROUP BY r.beta, m.mse, r.sxx;

CREATE VIEW

(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.225

slope,intercept
-0.03,-0.15


t_stat
-1.20


Although, holding all else constant, my ratings suggest a tendency to decrease about 0.03 stars relative to the Letterboxd average for every month that passes, my one-sample t-test of this coefficient yields a t-statistic of about -1.20, below the statistical significance at -1.96 at alpha = 0.05 significance level.

**Conclusion:** There is no conclusive evidence that I've gotten soft, but I should still be mindful of it because a t-statistic of -1.20 isn't necessarily negligible.

## **Question #2: How do my ratings compare with the rest of Letterboxd users?**

Time to backtrack a little from the last analysis on differences in ratings.

### **How do my overall ratings differ from others'?**

In [362]:
-- average difference between my rating and community rating
SELECT ROUND(AVG(my.myrating - lb.lb_rating)::NUMERIC, 2) avg_difference
FROM myratings my JOIN lbrating lb ON my.url_id = lb.url_id;

-- average absolute deviation of my rating from community rating
SELECT ROUND(AVG(ABS(my.myrating - lb.lb_rating))::NUMERIC, 2) avg_deviation
FROM myratings my JOIN lbrating lb ON my.url_id = lb.url_id;


(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.070

avg_difference
-0.20


avg_deviation
0.78


Despite my earlier conclusion that I might be rating films too favorably, it looks like my ratings are about 0.2 stars _lower_ than other users' ratings on average.

Further, my film ratings are an absolute average of 0.78 stars away from the community rating, which I say reflects a healthy amount of popular taste with sporadic contrarian takes. This raises an interesting follow-up question.

### **Which films did I have the hottest takes for?**

In [363]:
-- largest deviation in rating
SELECT my.title, ROUND((my.myrating - lb.lb_rating)::NUMERIC, 2) difference
FROM myratings my JOIN lbrating lb ON my.url_id = lb.url_id
ORDER BY ABS(my.myrating - lb.lb_rating) DESC
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.010

title,difference
Birdman or (The Unexpected Virtue of Ignorance),-3.49
Mulholland Drive,-3.24
Blade Runner 2049,-2.62
Rodrigo D. No Future,-2.59
"Black God, White Devil",-2.28
No,-2.28
Fight Club,-2.27
Paul Blart: Mall Cop,2.13
La hora de los hornos,-1.97
Tenet,-1.90


Of the ten films for which I differed from the Letterboxd average the most, **nine of them were negatively**! Hating Birdman, Mulholland Drive, and Blade Runner 2049 are apparently my hottest takes, which, while unsurprising, also indicates I have a thing aginst arthouse surrealism.

For completeness, here are the films I liked much more than most:

In [364]:
-- films I rated higher than the community average
SELECT my.title, ROUND((my.myrating - lb.lb_rating)::NUMERIC, 2) difference
FROM myratings my JOIN lbrating lb ON my.url_id = lb.url_id
ORDER BY difference DESC
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.014

title,difference
Paul Blart: Mall Cop,2.13
Home Alone 4,1.72
Teenage Mutant Ninja Turtles,1.66
The Karate Kid,1.54
Alvin and the Chipmunks,1.43
Jumanji: Welcome to the Jungle,1.42
Despicable Me 2,1.38
Wendy Wu: Homecoming Warrior,1.31
Borat Subsequent Moviefilm,1.22
Big Hero 6,1.22


Aside from exposing myself as a Paul Blart superfan, it looks like the majority of the films that I like more than most believe is warranted were those that I watched as a kid and have a special place in my heart.

### **How mainstream are my favorite films?**

To determine how basic or unconventional my film taste is, I examine how many ratings my 5 star films have on Letterboxd relative to all films I've rated.

In [365]:
-- Letterboxd rating count tertiles of my 5 star films relative to all films I have rated
WITH rc_tertiles (url_id, rating_count, tertile) AS (
    SELECT url_id, rating_count,
        NTILE(3) OVER (ORDER BY rating_count ASC)
    FROM ratingcount
    )
SELECT my.title, 
    rct.rating_count,
    CASE
        WHEN rct.tertile = 1 THEN 'Bottom Third'
        WHEN rct.tertile = 2 THEN 'Middle Third'
        ELSE 'Top Third'
    END rating_count_percentile
FROM myratings my JOIN rc_tertiles rct ON my.url_id = rct.url_id
WHERE my.myrating = 5
GROUP BY my.title, rct.rating_count, rct.tertile
ORDER BY rct.rating_count DESC;

(9 row(s) affected)

Total execution time: 00:00:00.103

title,rating_count,rating_count_percentile
Parasite,2910339,Top Third
Everything Everywhere All at Once,2401746,Top Third
La La Land,2363637,Top Third
Whiplash,2200199,Top Third
Eternal Sunshine of the Spotless Mind,1624872,Top Third
Challengers,1015597,Top Third
Big Hero 6,866794,Middle Third
Before Sunrise,721254,Middle Third
Nathan for You: Finding Frances,46219,Bottom Third


Regrettably, the majority of my favorite films have been watched and rated by high numbers in the Letterboxd community, so I can't justly claim the title of "film bro," not that I mind much.

**Conclusion:** Contrary to my self assessment, I appreciate mainstream and family films over arthouse deep cuts more than most Letterboxd users. But maybe that just speaks to the type of people who use Letterboxd.

## **Question #3: What characteristics do I like and dislike in films?**

### **What are my favorite and least favorite films?**

This is a reiteration of information seen above, but it helps to consolidate the films that represent my favorite cinema, and those that made my jaw drop for the wrong reasons:

In [367]:
-- films I have rated 5 stars
SELECT title, myrating
FROM myratings
WHERE myrating = 5
ORDER BY title ASC;

-- films I have rated 1 star or lower
SELECT title, myrating
FROM myratings
WHERE myrating <= 1
ORDER BY title ASC;

(9 row(s) affected)

(7 row(s) affected)

Total execution time: 00:00:00.069

title,myrating
Before Sunrise,5
Big Hero 6,5
Challengers,5
Eternal Sunshine of the Spotless Mind,5
Everything Everywhere All at Once,5
La La Land,5
Nathan for You: Finding Frances,5
Parasite,5
Whiplash,5


title,myrating
Birdman or (The Unexpected Virtue of Ignorance),0.5
Bulletproof Monk,1
Five Nights at Freddy's,1
Mulholland Drive,1
Rodrigo D. No Future,1
Superbabies: Baby Geniuses 2,1
The Emoji Movie,1


### <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"><b>Which directors do I enjoy the most?</b></span>

In [368]:
-- common directors among my favorably rated films
SELECT d.director, COUNT(d.director) director_count
FROM myratings my JOIN directors d ON my.url_id = d.url_id
WHERE my.myrating >= 4
GROUP BY d.director
HAVING COUNT(d.director) > 1
ORDER BY director_count DESC;

(9 row(s) affected)

Total execution time: 00:00:00.021

director,director_count
Christopher Nolan,4
Bong Joon-ho,3
Andrew Stanton,2
Steven Spielberg,2
Damien Chazelle,2
Rich Moore,2
Park Chan-wook,2
Matthew Vaughn,2
Chris Columbus,2


I'm not surprised to see **Christopher Nolan** and **Bong Joon-ho**, two of my favorite directors, as frequent appearances in my top films. But which directors have I actually given the highest ratings?

In [369]:
-- highest rated directors that have directed at least 2 films I rated
SELECT d.director, AVG(my.myrating) avg_rating
FROM myratings my JOIN directors d ON my.url_id = d.url_id
GROUP BY d.director
HAVING COUNT(*) >= 2 AND AVG(my.myrating) >= 4
ORDER BY avg_rating DESC;

(8 row(s) affected)

Total execution time: 00:00:00.006

director,avg_rating
Damien Chazelle,5
Matthew Vaughn,4.25
Bong Joon-ho,4.1
Andrew Stanton,4
Chris Columbus,4
Park Chan-wook,4
Pierre Coffin,4
Richard Linklater,4


Even though I've only seen two films by **Damien Chazelle**,  I absolutely loved both. This relation gives me more insight into directors I've consistently rated highly, and in conjunction with the first relation, I can identify directors who I enjoy but whose films I haven't seen too many of, such as **Damien Chazelle** and **Matthew Vaughn**.

### **What film genres do I enjoy the most?**

To understand which film genres I know I like to watch, I can see which ones appear the most among my favorite and least favorite films:

In [370]:
-- common genres among my favorably rated films
SELECT g.genre, COUNT(g.genre) genre_count
FROM myratings my JOIN genres g ON my.url_id = g.url_id
WHERE my.myrating >= 4
GROUP BY g.genre
ORDER BY genre_count DESC
LIMIT 5;

-- common genres among my unfavorably rated films
SELECT g.genre, COUNT(g.genre) genre_count
FROM myratings my JOIN genres g ON my.url_id = g.url_id
WHERE my.myrating <= 2
GROUP BY g.genre
ORDER BY genre_count DESC
LIMIT 5;

(5 row(s) affected)

(5 row(s) affected)

Total execution time: 00:00:00.105

genre,genre_count
Drama,37
Comedy,33
Adventure,32
Action,24
Family,21


genre,genre_count
Drama,13
Comedy,9
Action,6
Adventure,6
Fantasy,5


Although I can see that I enjoy the most **Drama** and **Comedy** films, it also looks like I dislike the most **Drama** and **Comedy** films! This may be a symptom of a large number of films just happening to fall in those categories, so instead I'll search for the genres that are **overrepresented** among my favorite and least favorite films, relative to their overall shares among films I've watched.

In [371]:
-- greatest difference from genre average share
CREATE OR REPLACE VIEW genre_shares AS
    SELECT DISTINCT g.genre, 
        ROUND((COUNT(g.genre)::NUMERIC / (
            SELECT COUNT(*) FROM myratings
            )
            ), 4) share
    FROM myratings my JOIN genres g ON my.url_id = g.url_id
    GROUP BY g.genre;

-- difference in genre share among highly rated films
SELECT gs.genre, (high_rating_share - share) diff_in_share
FROM genre_shares gs JOIN (
    SELECT DISTINCT g.genre, 
        ROUND(COUNT(g.genre)::NUMERIC / (
            SELECT COUNT(*) 
            FROM myratings 
            WHERE myrating >= 4
            ), 4) high_rating_share
    FROM myratings my JOIN genres g ON my.url_id = g.url_id
    WHERE my.myrating >= 4
    GROUP BY g.genre) g2 ON gs.genre = g2.genre
ORDER BY diff_in_share DESC
LIMIT 5;

-- difference in genre share among poorly rated films
SELECT gs.genre, (high_rating_share - share) diff_in_share
FROM genre_shares gs JOIN (
    SELECT DISTINCT g.genre, 
        ROUND((COUNT(g.genre)::NUMERIC / (
            SELECT COUNT(*) 
            FROM myratings 
            WHERE myrating <= 2)
            ), 4) high_rating_share
    FROM myratings my JOIN genres g ON my.url_id = g.url_id
    WHERE my.myrating <= 2
    GROUP BY g.genre) g2 ON gs.genre = g2.genre
ORDER BY diff_in_share DESC
LIMIT 5;

CREATE VIEW

(5 row(s) affected)

(5 row(s) affected)

Total execution time: 00:00:00.137

genre,diff_in_share
Adventure,0.0920
Family,0.0573
Comedy,0.0398
Animation,0.0245
Action,0.0178


genre,diff_in_share
Fantasy,0.0987
War,0.0652
Horror,0.0418
Western,0.0327
Documentary,0.0268


The first relation lists Adventure (9.20%), Family (5.73%), and Comedy (3.98%) films as those that appear in my favorably rated films at higher rates than across all my films watched, while the second reveals that Fantasy (9.87%), War (6.52%), and Horror (4.18%) genres are overrepresented. This provides a much clearer forecast: **seek out Adventure films**!

### **Which lesser-known studios produce films I enjoy?**

In [372]:
/* highest rated studios behind at least two films I have watched, averaging a rating above 
4 stars, and with average film rating count on Letterboxd below the average for all my films */
SELECT s.studio, ROUND(AVG(my.myrating)::NUMERIC, 2) avg_rating
FROM myratings my JOIN studios s ON my.url_id = s.url_id JOIN ratingcount rc ON my.url_id = rc.url_id
GROUP BY s.studio
HAVING COUNT(*) >= 2 AND AVG(my.myrating) > 4 AND AVG(rc.rating_count) < (SELECT AVG(rating_count) FROM ratingcount)
ORDER BY avg_rating DESC, s.studio ASC;

(4 row(s) affected)

Total execution time: 00:00:00.033

studio,avg_rating
Cloudy Productions,4.25
Marv,4.25
Matt Tolmach Productions,4.25
RatPac Entertainment,4.25


This relation displays film studios that I've rated multiple films for, average over four stars in my ratings, and have average film rating counts on Letterboxd below the average of all films I've rated. The four on this list represent smaller studios contributing to films I might want to explore!

### **Which decades do I think produced the best films?**

In [373]:
-- films watched and average rating by release decade
SELECT CONCAT(LEFT(year::VARCHAR, 3)::int * 10) decade, 
    COUNT(*) films_watched, 
    ROUND(AVG(myrating)::NUMERIC, 2) avg_rating
FROM myratings
GROUP BY LEFT(year::VARCHAR, 3)::int * 10
ORDER BY decade ASC;

(7 row(s) affected)

Total execution time: 00:00:00.005

decade,movies_watched,avg_rating
1950,2,3.75
1960,2,1.75
1980,5,3.00
1990,21,3.29
2000,33,3.50
2010,80,3.49
2020,28,3.54


My first takeaway from this relation is that I've watched a high number of films released in the 2010s and its neighboring decades, but **very few from before 1990** — I should watch more films from the 80s and earlier to round out my film taste. That said, this drought may be for good reason: overall, I tend to give films from **2000 and later** higher ratings than those from before. I'll take this as a sign to overcome my bias against older films and find some that I like!

### **Do I have any favorite actors?**

I've never considered myself someone who follows celebrities closely or considers who the lead actor is when picking a film to watch. But are there actors I gravitate towards without realizing?

In [374]:
-- highest rated actors in at least three films I rated
SELECT a.actor, 
    COUNT(a.actor) appearances, ROUND(AVG(my.myrating)::NUMERIC, 2) avg_rating
FROM myratings my JOIN actors a ON my.url_id = a.url_id
GROUP BY a.actor
HAVING COUNT(*) >= 3
ORDER BY avg_rating DESC
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.059

actor,appearances,avg_rating
Joseph Oliveira,3,4.67
Jenny Slate,3,4.50
Lee Jung-eun,3,4.50
Choi Woo-shik,3,4.33
Colin Firth,3,4.33
Ahn Seong-bong,3,4.33
Lee Ji-hye,3,4.33
Yuri Lowenthal,3,4.33
J.K. Simmons,5,4.30
David Cowgill,4,4.25


On this list are some familiar names like **Jenny Slate**, **Colin Firth**, and **J.K. Simmons** — now I have a few names to search for other films they've been a part of. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">But who are </span> **Joseph Oliveira** <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">(the top of my list), </span> **Lee Jung-eun** <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">(in films with an impressive 4.5 average rating), and </span> **David Cowgill** <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">(with four appearances)?</span>

Google tells me that Oliveira is a prolific extra in Christopher Nolan and MCU films, Lee is an actress who frequently collaborates with Bong Joon-ho, and David Cowgill is a voice actor for numerous animated films. Props to them for contributing semi-anonymously to the projects I love!

### **Do I have a preference for long vs. short films?**

In [375]:
-- directors producing long films and my average rating of them
SELECT d.director, ROUND(AVG(l.length)::NUMERIC, 2) avg_length, 
    ROUND(AVG(my.myrating)::NUMERIC, 2) avg_rating
FROM directors d JOIN length l ON d.url_id = l.url_id JOIN myratings my ON l.url_id = my.url_id
GROUP BY d.director
HAVING COUNT(d.*) >= 2
ORDER BY avg_length DESC
LIMIT 5;

-- directors producing short films and my average rating of them
SELECT d.director, ROUND(AVG(l.length)::NUMERIC, 2) avg_length, 
    ROUND(AVG(my.myrating)::NUMERIC, 2) avg_rating
FROM directors d JOIN length l ON d.url_id = l.url_id JOIN myratings my ON l.url_id = my.url_id
GROUP BY d.director
HAVING COUNT(d.*) >= 2
ORDER BY avg_length ASC
LIMIT 5;

(5 row(s) affected)

(5 row(s) affected)

Total execution time: 00:00:00.088

director,avg_length,avg_rating
Joe Russo,159.00,3.50
Anthony Russo,159.00,3.50
Christopher Nolan,146.11,3.28
Jon Watts,140.50,3.50
Steven Spielberg,137.00,3.38


director,avg_length,avg_rating
Conrad Vernon,91.00,2.25
Pierre Coffin,94.50,4.00
Julia Solomonoff,95.00,2.75
Richard Linklater,96.67,4.00
Andrew Stanton,97.50,4.00


From these relations, it's hard to tell if I rate directors of longer films higher or lower than directors of shorter films — I do notice that the variability in my ratings is greater for the latter.

I'll perform another linear regression of my ratings on film length in minutes, and a t-test of the coefficient on the length coefficient to determine if it significantly boosts or hurts my evaluations:

In [376]:
-- linear regression of my film ratings on film length
SELECT ROUND(REGR_SLOPE(my.myrating, l.length)::NUMERIC, 2) slope, 
    ROUND(REGR_INTERCEPT(my.myrating, l.length)::NUMERIC, 2) intercept
FROM myratings my JOIN length l ON l.url_id = my.url_id;

-- t-test for significance of length coefficient, assuming normal sampling distribution given n >= 30
WITH regression (beta, alpha, sxx) AS (
    SELECT REGR_SLOPE(my.myrating, l.length), 
        REGR_INTERCEPT(my.myrating, l.length),
        REGR_SXX(my.myrating, l.length)
    FROM myratings my JOIN length l ON l.url_id = my.url_id
    ),
mean_squared_error (mse) AS (
    SELECT SUM(rss.res_sum_squares) / (COUNT(*) - 2)
    FROM (
        SELECT POWER(my.myrating - (r.beta * l.length + r.alpha), 2) res_sum_squares
        FROM myratings my JOIN length l ON l.url_id = my.url_id, regression r
        ) rss
    )
SELECT ROUND((r.beta / SQRT(m.mse / r.sxx))::NUMERIC, 2) t_stat
FROM regression r, mean_squared_error m
GROUP BY r.beta, m.mse, r.sxx;

(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.042

slope,intercept
0.00,3.42


t_stat
0.10573921165047181


Contrary to the last regression, the t-statistic of 0.11 is not even close to the 1.96 threshold for significance at alpha = 0.05. There is no evidence that my ratings depend on how long or short a film it. Even if the effect was significant, a 30 minute difference would be expected to raise my review by only 0.0057 stars.

**Conclusion:** I can enjoy both longer and shorter films!

## **Final Thoughts**

From this data analysis, I've answered my three questions soundly:

**1\. Am I rating films regularly and reliably?**

Overall, yes! I had a few droughts this year where I didn't watch films as frequently as I wanted to, but my ratings are relatively spread out, and I concluded that there's no reason to believe my ratings deviated in fairness over time.

**2\. How do my film ratings compare with the Letterboxd community at large?**

I tend to be a little more critical than the average Letterboxd user, but within the realm of typical variation in film taste. I rated many family films much higher than average, and <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">arthouse films much lower than average. Even so, my favorite films are relatively mainstream.</span>

**3\. Can my ratings reveal characteristics of films that I should seek out — or avoid — when finding new films to watch?**

I reaffirmed my love for directors Damien Chazelle, Bong Joon-ho, and Christopher Nolan, but I revealed several less popular directors, studios, and actors who I plan to watch more of in the future. I tend to enjoy Adventure and Family films, and dislike Fantasy and War films. Lastly, I can get behind both longer films and shorter films without problem.
